In [1]:
import cobra
import pandas as pd
import re
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
from pathlib import Path
import sys
sys.path.append('../../code/')
import leakage, utils
import pubchempy as pcp
import seaborn as sns
import time


# Settings / choices

In [2]:
only_significant_changes = False
timepoints = np.arange(1.5, 17, 1)
knock_outs = False
shadow_price_for_leaked_mets = True
species = 'b_licheniformis'

In [3]:
model_fn = '../../models/{0}/iBsu1147_irr_enz_constraint_adj.xml'.format(species)
model = cobra.io.read_sbml_model(model_fn)
# model = cobra.io.load_matlab_model('../../models/{0}/ec_iYO844.mat'.format(species))

model.solver = 'gurobi'

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [4]:
model.optimize()
print(model.summary())

Objective
1.0 bio00006 = 0.37536765800300287

Uptake
------
Metabolite    Reaction     Flux  C-Number  C-Flux
     ca2_e    EX_ca2_e  0.00112         0   0.00%
     fe3_e    EX_fe3_e 0.001205         0   0.00%
  glc__D_e EX_glc__D_e        5         6 100.00%
       k_e      EX_k_e   0.2468         0   0.00%
     mg2_e    EX_mg2_e  0.03556         0   0.00%
     nh4_e    EX_nh4_e    3.384         0   0.00%
      o2_e     EX_o2_e    14.28         0   0.00%
      pi_e     EX_pi_e   0.3832         0   0.00%
     so4_e    EX_so4_e  0.07438         0   0.00%
 prot_pool enzyme_pool    0.165         0   0.00%

Secretion
---------
Metabolite Reaction   Flux  C-Number  C-Flux
     co2_e EX_co2_e -15.21         1 100.00%
     h2o_e EX_h2o_e -23.63         0   0.00%
       h_e   EX_h_e -2.813         0   0.00%



In [5]:
def get_leakage(time):
    exometabolites_folder = Path("../../data/{0}/".format(species))
    leakage_df = leakage.get_leakage(exometabolites_folder, species, time = time, unit = '/gDW', method = 'one-way-diff',
                                    only_significant_changes = only_significant_changes)
    leakage_df.set_index("Metabolite", inplace=True)
    leakage_df.drop_duplicates(inplace=True)
    leakage_label = "Leakage (mmol/gDW/h)"
    return leakage_df

In [6]:
exometabolites_folder = Path("../../data/b_licheniformis")


In [7]:
met_info_df = pd.read_csv("../../data/met_info_curated.csv", encoding = "ISO-8859-1", index_col = 0)

In [8]:
# Read metabolite mapping
mapping_df = pd.read_csv('../../data/id_mapping.csv', index_col=0)
# df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
# df2.drop(columns='Metabolite name', inplace=True)

# Get leakage


In [9]:
# timepoints = np.arange(1.5, 12, 1)#[5,6,7,8,9,10,11, 12, 13]
for i, t in enumerate(timepoints):
    print(t)
    leakage_df = get_leakage(t)
    # Consider to use an earlier time-point
    glucose_uptake_rate = leakage.get_glucose_uptake_rate(exometabolites_folder, species, time = t, method = 'one-way-diff')
    df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
    df2.drop(columns='Metabolite name', inplace=True)
    df = pd.merge(met_info_df, df2, left_on = 'Metabolite id', right_on = 'Bacillus metabolite')
    df['Time'] = t
    df['Glucose'] = -glucose_uptake_rate
    print(glucose_uptake_rate)
    # Set model constraints
    model = cobra.io.read_sbml_model(model_fn)
    with model:
        model.reactions.EX_glc__D_e.lower_bound = min(glucose_uptake_rate, 0)
        print(model.slim_optimize())
        for j, row in df.iterrows():
            if row['Leakage (mmol/gDW/h)'] < 0:
                met_ids = row['Metabolite id'].split(',')
                mets = []
                for m_id in met_ids:
                    try:
                        m = model.metabolites.get_by_id('{0}_e'.format(m_id.strip(' ')))
                    except KeyError:
                        continue
                    else:
                        mets.append(m)
                for m in mets:
                    r_ex = [r for r in m.reactions if len(r.metabolites)==1][0]
                    x = model.slim_optimize()
                    r_ex.lower_bound = row['Leakage (mmol/gDW/h)']/len(mets)
                    # print(m.id, x, model.slim_optimize(), r_ex.bounds)
                    # Should check soplutions
                    # print(r_ex.id)
        solution = model.optimize()
        # print(model.summary())
        try:
            pfba_solution = cobra.flux_analysis.pfba(model.copy())
        except:
            print('No feasible model at t: ', t)
            continue
        print(model.slim_optimize())
        # List already excreted metabolites
        exchanged_mets = {}
        for r in model.boundary:
            flux = solution.fluxes[r.id]
            if flux != 0:
                exchanged_mets[list(r.metabolites.keys())[0].id[:-2]]=(r.id, flux)
        
        df['Predicted growth rate'] = solution.objective_value
        # Get turnover and shadow prices
        turnover = {}
        shadow_prices = {}        
        for j, row in df.iterrows():
            if row['Leakage (mmol/gDW/h)'] > 0:
                met_ids = row['Metabolite id'].split(',')
                sp_list = []
                turnover_list = []
                for key in met_ids:
                    if key.strip() in exchanged_mets.keys():
                        if shadow_price_for_leaked_mets:
                            # the change should be taken into account the already exchanged flux
                            existing_flux = exchanged_mets[key.strip()]
                        else:
                            continue
                    else:
                        existing_flux = None
                    m_id = "{0}_c".format(key.strip())
                    m = model.metabolites.get_by_id(m_id)
                    # x = solution.objective_value
                    # x2 = model.optimize().objective_value
                    sp_ji = leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.01, existing_flux = existing_flux)
                    if np.abs(sp_ji-solution.shadow_prices[m.id]) > 0.01:
                        sp_ji = solution.shadow_prices[m.id]
                        # Need to figure out what's wrong here
                            # solution = M.optimize()
                        print("######")
                            # print(x, x2, solution.objective_value)
                            # print(m.id, sp_ji, solution.shadow_prices[m.id], existing_flux)
                    sp_list.append(sp_ji)
                    # turnover_list.append(get_turnover_flux(m, pfba_solution))
                    turnover_list.append(m.summary(pfba_solution).producing_flux['flux'].sum())
                # print(met_ids, sp_list, turnover_list)
                # Shadow prices
                if len(sp_list):
                    shadow_prices[j] = np.nanmean(sp_list)
                    turnover[j] = np.mean(turnover_list)
                else:
                    shadow_prices[j] = np.nan
                    turnover[j] = np.nan
                # print(met_ids, np.nanmean(sp_list))
        df["Shadow price"] = pd.Series(shadow_prices)
        df["Turnover"] = pd.Series(turnover)
        
    if i == 0:
        full_df = df
    else:
        full_df = pd.concat([full_df, df])


1.5
-0.8078995160054667
0.047394887329702415
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpzdrtw06d.lp
Reading time = 0.02 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.047526561857171905
2.5
-6.093979669035206
0.4578926415354851
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpq3xr8trq.lp
Reading time = 0.01 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.4593899312143562
3.5
-6.435646272425405
0.4836664576923116
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpcmaprvn1.lp
Reading time = 0.02 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.4840899275284319
4.5
-6.2710922627098995
0.4712532315859496
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmp2v_y_67b.lp
Reading time = 0.02 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.472006227156904
5.5
-3.868136553589917
0.2899848899846236
Read LP format model from file /var/folders/h6/4b

/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


No feasible model at t:  10.5
11.5
0.0
nan
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpul4a7m1j.lp
Reading time = 0.02 seconds
: 1460 rows, 6592 columns, 32868 nonzeros


/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


No feasible model at t:  11.5
12.5
0.0
nan
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmp33ipypaw.lp
Reading time = 0.01 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.006558295769946788
13.5
0.0
nan
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpxe1oifyr.lp
Reading time = 0.01 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.02188386070341866
14.5
0.0
nan
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmp6n098_ux.lp
Reading time = 0.02 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.0328058024990594
15.5
0.0
nan
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmp2pne67u6.lp
Reading time = 0.01 seconds
: 1460 rows, 6592 columns, 32868 nonzeros
0.01597654488737131
16.5
0.0
nan
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpxip9bthv.lp
Reading time = 0.02 seconds
: 1460 rows, 6592 columns, 32868 nonzeros


/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


No feasible model at t:  16.5


In [10]:
full_df

,Metabolite name,Metabolite id,Value,Uncertainty,Mass,Charge,Phosphate,Topological Polar Surface Area [],Concentration,log P,Leakage (mmol/gDW/h),Ecoli metabolite,Yeast metabolite,Bacillus metabolite,Time,Glucose,Predicted growth rate,Shadow price,Turnover
0,Dihydroxyacetonephosphate,dhap,50.40,1.74,168.041961,-2.0,1,104.0,0.000374,-2.5,0.001493,dhap,s_0629,dhap,1.5,0.8079,0.047527,-0.046907,0.759988
1,Glyceraldehyde-3-phosphate,g3p,15.04,0.24,168.041961,-2.0,1,104.0,NaN,-2.7,0.000176,g3p,s_0764,g3p,1.5,0.8079,0.047527,-0.046907,1.494194
2,2/3-phosphoglycerate,"2pg, 3pg",19.45,1.55,183.033421,-3.0,1,124.0,0.001540,-2.6,0.004842,"2pg, 3pg","s_0188, s_0260","2pg, 3pg",1.5,0.8079,0.047527,-0.038531,0.982294
3,Phosphoenolpyruvate,pep,20.43,0.28,165.018141,-3.0,1,104.0,0.000184,-1.1,0.002034,pep,s_1360,pep,1.5,0.8079,0.047527,-0.038531,0.493572
4,Pyruvate,pyr,4924.00,509.00,87.054120,-1.0,0,57.2,NaN,-0.6,0.089897,pyr,s_1399,pyr,1.5,0.8079,0.047527,-0.032668,3.133064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,Arginine,arg__L,9.61,0.91,175.208900,1.0,0,128.0,0.000569,-4.2,-0.000035,arg__L,s_0965,arg__L,15.5,-0.0000,0.015977,NaN,NaN
22,Histidine,his__L,6.84,0.01,155.154560,0.0,0,92.0,0.000068,-3.2,0.000673,his__L,s_1006,his__L,15.5,-0.0000,0.015977,-0.103493,0.001518
23,Acetate,ac,12949.00,851.00,59.044020,-1.0,0,40.1,NaN,0.4,-1.747695,ac,s_0362,ac,15.5,-0.0000,0.015977,NaN,NaN
24,Orotate,orot,1100.00,27.00,155.088320,-1.0,0,98.3,NaN,-0.8,-0.000640,orot,s_1269,orot,15.5,-0.0000,0.015977,NaN,NaN


# Add median shadow prices


In [11]:
# full_leakage['Uptake (mmol/gDW/h)'] = 0
new_df = full_df.copy()

In [12]:
median_sp_df = pd.read_csv('../../results/b_licheniformis/median_sp.csv', index_col = 0)

In [13]:
median_sp = []
low_sp = []
high_sp = []
for i, row in new_df.iterrows():
    
    gr = np.round(row['Predicted growth rate'], 1)
    
    tmp_m = []
    tmp_low = []
    tmp_high = []
    for key in row['Ecoli metabolite'].split(','):
        met_id = '{0}_c'.format(key.strip())
        met_df = median_sp_df.loc[met_id,:]
        tmp_m.append(met_df.loc[met_df['Growth rate']==gr, 'log10(-Shadow price)'].values[0])
        tmp_low.append(met_df.loc[met_df['Growth rate']==0.2, 'log10(-Shadow price)'].values[0])
        tmp_high.append(met_df.loc[met_df['Growth rate']==0.5, 'log10(-Shadow price)'].values[0])
    median_sp.append(np.mean(tmp_m))
    low_sp.append(np.mean(tmp_low))
    high_sp.append(np.mean(tmp_high))

In [14]:
new_df['Median log10(-Shadow price)'] = median_sp
new_df['Low log10(-Shadow price)'] = low_sp
new_df['High log10(-Shadow price)'] = high_sp

In [15]:
# np.round?

In [16]:
# full_leakage['Uptake (mmol/gDW/h)'] = 0
# new_df = full_df.copy()

In [17]:
new_df['Uptake (mmol/gDW/h)'] = -1*new_df['Leakage (mmol/gDW/h)']

In [18]:
new_df.loc[new_df['Leakage (mmol/gDW/h)'] < 0, 'Leakage (mmol/gDW/h)'] = np.nan
new_df.loc[new_df['Uptake (mmol/gDW/h)'] < 0, 'Uptake (mmol/gDW/h)'] = 0


In [19]:
new_df['log10(Leakage [mmol/gDW/h])'] = np.log10(new_df['Leakage (mmol/gDW/h)'])
new_df['log10(-Shadow price [gDW/mmol])'] = np.log10(-new_df['Shadow price'])
new_df['log10(Turnover [mmol/gDW/h])'] = np.log10(new_df['Turnover']).replace(-np.inf, np.nan)

/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
new_df.drop(columns=['Value', 'Uncertainty', 'Yeast metabolite', 'Ecoli metabolite', 'Bacillus metabolite'], inplace = True)

In [21]:
timestr = time.strftime("%Y%m%d")

if only_significant_changes:
    s1 = '_osc'
else:
    s1 = ''

if shadow_price_for_leaked_mets:
    s2 = '_SP_for_leaked'
else:
    s2 = ''

if knock_outs:
    s3 = '_KO'
else:
    s3 = ''
fn = 'spreadsheet_{0}_leakage_{1}{2}{3}{4}.csv'.format(species, timestr, s1, s2, s3)
folder = Path('../../results/{0}/'.format(species))
new_df.to_csv(folder / fn)

In [22]:
# full_leakage = full_df.loc[~full_df['Shadow price'].isna(), :]
# full_leakage = full_leakage.loc[full_leakage.Turnover <100, :]
# full_leakage = full_leakage.loc[full_leakage['Leakage (mmol/gDW/h)'] > 0, :]

In [23]:
# full_leakage['log10(leakage)'] = np.log10(full_leakage['Leakage (mmol/gDW/h)'])
# full_leakage['log10(-Shadow price)'] = np.log10(-full_leakage['Shadow price'])
# full_leakage['log10(Turnover)'] = np.log10(full_leakage['Turnover'])